In [1]:
import git
from git import Repo

from unidiff import PatchSet

import subprocess
import json
import pandas as pd
import numpy as np
import matplotlib
import sys
import matplotlib.pyplot as plt
import sklearn

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.ensemble import ExtraTreesClassifier

In [2]:
mutants_and_tests = pd.read_pickle('flask_full_with_edit_distances.pkl')
len(mutants_and_tests)
display(mutants_and_tests)
print(mutants_and_tests.columns)

,mutant_id,current_line,line_number_changed,modified_file_path,modified_method,previous_line,repo_path,outcome,test_id,full_name,name,filepath,duration,setup_outcome,setup_duration,call_outcome,call_duration,teardown_outcome,teardown_duration,edit_distance_method_name_test_name
0,1873.0,debug = None,841.0,src/flask/cli.py,run_command,,/tmp/tmpszy15usp,True,0.0,tests/test_appctx.py::test_basic_url_generation,test_basic_url_generation,tests/test_appctx.py,0.059266,True,0.029150,True,0.000706,True,0.000260,11
1,1873.0,debug = None,841.0,src/flask/cli.py,run_command,,/tmp/tmpszy15usp,True,1.0,tests/test_appctx.py::test_url_generation_requ...,test_url_generation_requires_server_name,tests/test_appctx.py,0.003147,True,0.001320,True,0.000298,True,0.000208,11
2,1873.0,debug = None,841.0,src/flask/cli.py,run_command,,/tmp/tmpszy15usp,True,2.0,tests/test_appctx.py::test_url_generation_with...,test_url_generation_without_context_fails,tests/test_appctx.py,0.000988,True,0.000293,True,0.000176,True,0.000225,11
3,1873.0,debug = None,841.0,src/flask/cli.py,run_command,,/tmp/tmpszy15usp,True,3.0,tests/test_appctx.py::test_request_context_mea...,test_request_context_means_app_context,tests/test_appctx.py,0.003678,True,0.001334,True,0.000801,True,0.000208,11
4,1873.0,debug = None,841.0,src/flask/cli.py,run_command,,/tmp/tmpszy15usp,True,4.0,tests/test_appctx.py::test_app_context_provide...,test_app_context_provides_current_app,tests/test_appctx.py,0.003005,True,0.001300,True,0.000202,True,0.000202,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
958036,1040.0,if self.session_interface.is_null_sess...,2174.0,src/flask/app.py,process_response,,/tmp/tmpsuxyyqwi,True,463.0,tests/test_views.py::test_explicit_head,test_explicit_head,tests/test_views.py,0.005060,True,0.001630,True,0.001569,True,0.000230,16
958037,1040.0,if self.session_interface.is_null_sess...,2174.0,src/flask/app.py,process_response,,/tmp/tmpsuxyyqwi,True,464.0,tests/test_views.py::test_endpoint_override,test_endpoint_override,tests/test_views.py,0.005697,True,0.001254,True,0.002970,True,0.000220,16
958038,1040.0,if self.session_interface.is_null_sess...,2174.0,src/flask/app.py,process_response,,/tmp/tmpsuxyyqwi,True,465.0,tests/test_views.py::test_methods_var_inheritance,test_methods_var_inheritance,tests/test_views.py,0.004606,True,0.001403,True,0.001572,True,0.000228,16
958039,1040.0,if self.session_interface.is_null_sess...,2174.0,src/flask/app.py,process_response,,/tmp/tmpsuxyyqwi,True,466.0,tests/test_views.py::test_multiple_inheritance,test_multiple_inheritance,tests/test_views.py,0.004516,True,0.001343,True,0.001601,True,0.000228,16


Index(['mutant_id', 'current_line', 'line_number_changed',
       'modified_file_path', 'modified_method', 'previous_line', 'repo_path',
       'outcome', 'test_id', 'full_name', 'name', 'filepath', 'duration',
       'setup_outcome', 'setup_duration', 'call_outcome', 'call_duration',
       'teardown_outcome', 'teardown_duration',
       'edit_distance_method_name_test_name'],
      dtype='object')


# Data Preparation

In [9]:
feature_columns = ["mutant_id", "edit_distance_method_name_test_name", "current_line", "line_number_changed", "modified_file_path", "previous_line", "test_id", "name", "filepath", "modified_method"]
#feature_columns = ["current_line", "line_number_changed", "modified_file_path", "previous_line", "test_id", "name", "filepath"]
X = mutants_and_tests[feature_columns]
y = mutants_and_tests["outcome"]

# Throw out nan values:
X = X.loc[mutants_and_tests[feature_columns].isnull().any(axis=1) == False]
y = mutants_and_tests["outcome"].loc[mutants_and_tests[feature_columns].isnull().any(axis=1) == False]
y = y.astype('bool')
len(X)


920088

In [10]:
# Encode stuff we want to encode:
encoded_columns = ["modified_method", "modified_file_path", "name", "filepath", "current_line", "previous_line"]
# encoded_columns = ["name", "filepath", "modified_file_path", "current_line", "previous_line"]
# Encoding
from sklearn.preprocessing import OrdinalEncoder
enc = OrdinalEncoder()
X_enc = enc.fit_transform(X[encoded_columns])
i = 0
for column_name in encoded_columns:
    X[column_name] = X_enc[:,i]
    i += 1
X

,mutant_id,edit_distance_method_name_test_name,current_line,line_number_changed,modified_file_path,previous_line,test_id,name,filepath,modified_method
0,1873.0,11,1307.0,841.0,2.0,0.0,0.0,116.0,0.0,189.0
1,1873.0,11,1307.0,841.0,2.0,0.0,1.0,602.0,0.0,189.0
2,1873.0,11,1307.0,841.0,2.0,0.0,2.0,603.0,0.0,189.0
3,1873.0,11,1307.0,841.0,2.0,0.0,3.0,501.0,0.0,189.0
4,1873.0,11,1307.0,841.0,2.0,0.0,4.0,102.0,0.0,189.0
...,...,...,...,...,...,...,...,...,...,...
958036,1040.0,16,766.0,2174.0,0.0,0.0,463.0,216.0,17.0,175.0
958037,1040.0,16,766.0,2174.0,0.0,0.0,464.0,196.0,17.0,175.0
958038,1040.0,16,766.0,2174.0,0.0,0.0,465.0,278.0,17.0,175.0
958039,1040.0,16,766.0,2174.0,0.0,0.0,466.0,282.0,17.0,175.0


In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=420, shuffle=False)
X_train

,mutant_id,edit_distance_method_name_test_name,current_line,line_number_changed,modified_file_path,previous_line,test_id,name,filepath,modified_method
0,1873.0,11,1307.0,841.0,2.0,0.0,0.0,116.0,0.0,189.0
1,1873.0,11,1307.0,841.0,2.0,0.0,1.0,602.0,0.0,189.0
2,1873.0,11,1307.0,841.0,2.0,0.0,2.0,603.0,0.0,189.0
3,1873.0,11,1307.0,841.0,2.0,0.0,3.0,501.0,0.0,189.0
4,1873.0,11,1307.0,841.0,2.0,0.0,4.0,102.0,0.0,189.0
...,...,...,...,...,...,...,...,...,...,...
650640,1206.0,9,813.0,133.0,7.0,0.0,97.0,627.0,1.0,111.0
650641,1206.0,9,813.0,133.0,7.0,0.0,98.0,618.0,1.0,111.0
650642,1206.0,9,813.0,133.0,7.0,0.0,99.0,617.0,1.0,111.0
650643,1206.0,9,813.0,133.0,7.0,0.0,100.0,616.0,1.0,111.0


In [14]:
X_test

,mutant_id,edit_distance_method_name_test_name,current_line,line_number_changed,modified_file_path,previous_line,test_id,name,filepath,modified_method
650645,1206.0,9,813.0,133.0,7.0,0.0,102.0,614.0,1.0,111.0
650646,1206.0,9,813.0,133.0,7.0,0.0,103.0,613.0,1.0,111.0
650647,1206.0,9,813.0,133.0,7.0,0.0,104.0,612.0,1.0,111.0
650648,1206.0,9,813.0,133.0,7.0,0.0,105.0,611.0,1.0,111.0
650649,1206.0,9,813.0,133.0,7.0,0.0,106.0,274.0,1.0,111.0
...,...,...,...,...,...,...,...,...,...,...
958036,1040.0,16,766.0,2174.0,0.0,0.0,463.0,216.0,17.0,175.0
958037,1040.0,16,766.0,2174.0,0.0,0.0,464.0,196.0,17.0,175.0
958038,1040.0,16,766.0,2174.0,0.0,0.0,465.0,278.0,17.0,175.0
958039,1040.0,16,766.0,2174.0,0.0,0.0,466.0,282.0,17.0,175.0


In [32]:
# Baseline: Search for the mutant with the smallest distance, and predict the same values


print(y_train.value_counts())
print(y_test.value_counts())

predictions = []

for index, row in X_test.iterrows():
    
    # Select only rows from X_train with the same test_id
    correct_tests = X_train.loc[X_train['test_id'] == row['test_id']]
    #print("Test_id: " + str(row['test_id']))
    #print("Mutant_id: " + str(row['mutant_id']))
    #print(len(correct_tests))
    mutant_id = row['mutant_id']
    #print(correct_tests)
    nearest_mutant_id_index = abs(correct_tests['mutant_id'] - mutant_id).idxmin()
    #print(nearest_mutant_id_index)
    #print(y_train)
    predictions.append(y_train[nearest_mutant_id_index])



print(index)

print(len(predictions))

print(classification_report(y_test, predictions))

True     573303
False     43155
Name: outcome, dtype: int64
True     265942
False     37688
Name: outcome, dtype: int64
958040
303630
              precision    recall  f1-score   support

       False       0.66      0.29      0.40     37688
        True       0.91      0.98      0.94    265942

    accuracy                           0.89    303630
   macro avg       0.78      0.63      0.67    303630
weighted avg       0.88      0.89      0.87    303630



In [18]:
y_test

650645    True
650646    True
650647    True
650648    True
650649    True
          ... 
958036    True
958037    True
958038    True
958039    True
958040    True
Name: outcome, Length: 303630, dtype: bool